In [ ]:
from matplotlib.colors import LinearSegmentedColormap
import matplotlib.pyplot as plt
import seaborn as sns
import future.utils
import numpy as np
import ndlib.models.ModelConfig as mc
import ndlib.models.opinions as op
import networkx as nx
import os
import json
import gzip
from scipy import stats

: 

In [ ]:
g = nx.read_edgelist('euro2020/euro2020_edgelist.csv', delimiter=',', nodetype=str, )

: 

In [ ]:
g

: 

In [ ]:
g.number_of_nodes()

: 

In [ ]:
g.number_of_edges()

: 

In [ ]:
mapping = dict()
i = 0
for nodename in g.nodes():
    mapping[nodename] = i
    i+=1

with open('euro2020/node_mapping.json', 'w') as f:
    json.dump(mapping, f)

graph = nx.relabel_nodes(g, mapping)

graph.nodes()

: 

In [ ]:
with open('euro2020/euro2020_t0.json', 'r') as f:
    nodelist = json.load(f)

: 

In [ ]:
nodes = {}
for k, v in nodelist.items():
    try:
        nodes[mapping[k]] = float(v)
    except KeyError:
        print(f'node {k} not present in graph')
        continue

: 

In [ ]:
type(list(nodes.values())[0])

: 

In [65]:
print(len(nodes))

2925


In [66]:
pros = {k:v for k,v in nodes.items() if v <= 0.4}
pros

avg_pros = np.average(np.array(list(pros.values())))
avg_pros

0.2825807341574699

In [67]:
cons = {k:v for k,v in nodes.items() if v >= 0.6}
cons

avg_cons = np.average(np.array(list(cons.values())))
avg_cons

0.865708635554234

In [68]:
neut = {k:v for k,v in nodes.items() if v < 0.6 and v > 0.4}
neut

avg_neut = np.average(np.array(list(neut.values())))
avg_neut

0.48528938156222684

In [69]:
import csv
openmindedness = {}
with open('euro2020/euro2020_openMindedness.csv', 'r') as f:
    csv_reader = csv.reader(f)
    next(csv_reader)
    for row in csv_reader:
        try:
            openmindedness[mapping[row[1]]] = float(row[2])
        except KeyError:
            print('node not present in graph')
            continue
openmindedness

{451: 0.0,
 2368: 0.0,
 2480: 0.0,
 2814: 1.0,
 2582: 0.0,
 791: 0.1467803030303031,
 573: 0.382716049382716,
 350: 0.08510101010101001,
 224: 0.0,
 1778: 0.0,
 1671: 0.12070707070707072,
 2758: 0.0,
 2443: 0.0,
 1444: 0.3055555555555556,
 2630: 0.0,
 759: 0.09999999999999998,
 2643: 0.3333333333333333,
 831: 0.16666666666666666,
 716: 0.0,
 1873: 0.0,
 478: 0.06862745098039225,
 1251: 0.03333333333333338,
 1051: 0.17361111111111116,
 134: 0.11111111111111104,
 2735: 0.0,
 2811: 0.0,
 188: 0.09999999999999998,
 316: 0.0,
 306: 0.0,
 1034: 0.29629629629629634,
 2725: 0.3888888888888889,
 2822: 0.16666666666666666,
 1728: 0.34920634920634924,
 1477: 0.04545454545454541,
 366: 0.5,
 1018: 0.5059259259259259,
 1886: 0.16666666666666666,
 427: 0.16666666666666674,
 1941: 0.3333333333333333,
 1321: 0.2500000000000001,
 428: 0.23919753086419754,
 483: 0.1333333333333333,
 2185: 0.0,
 1844: 0.6666666666666667,
 1946: 0.5,
 1084: 0.0,
 2540: 0.0,
 1219: 0.16666666666666669,
 2499: 0.0,
 571: 0.

In [70]:
type(list(graph.nodes)[0])

int

In [71]:
from utils import *
from aggregate import *
from plots import *
import sys

# Model settings
media_opinions = [[avg_cons], [avg_pros], [avg_neut]]

max_it = 100000
    
gammas, pms = [0.0, 0.5, 1.0, 1.5], [0.5]
#perform multiple runs and average results
for media_op in media_opinions:
    k = len(media_op)
    for gamma in gammas:
        for pm in pms:
            epsilon = 'heterogeneous'
            respath = 'res/'
            name = f'e{epsilon}_g{gamma}_pm{pm}_mo{media_op}'
            final_opinions, final_niter = read_dicts(respath, name, max_it)

            for run in range(1):
                if str(run) in final_opinions.keys() and str(run) in final_niter.keys(): 
                    print('run already present. skipping.')
                    run += 1
                else:
                    print(f'run {run} epsilon = {epsilon} gamma = {gamma} pm = {pm} media_op = {media_op}')    

                    #create model
                    model = op.AlgorithmicBiasMediaModel(graph)

                    #create configuration
                    config = mc.Configuration()
                    config.add_model_parameter("mu", 0.5)
                    config.add_model_parameter("gamma", gamma)
                    config.add_model_parameter("gamma_media", gamma)
                    config.add_model_parameter("p", pm)
                    config.add_model_parameter("k", k)

                    for node in list(graph.nodes):
                        config.add_node_configuration("epsilon_node", node, float(openmindedness[node]))

                    #configure model
                    model.set_initial_status(configuration=config, initial_status=nodes)
                    model.set_media_opinions(media_op)

                    #perform iterations untill convergence
                    iterations = model.steady_state(max_iterations=max_it, nsteady=1000, sensibility=0.001, node_status=True, progress_bar=True)

#                     with gzip.open(respath+f'iterations_{name}_maxit{max_it}.json.gz', 'wb') as ofile:
#                         ofile.write(json.dumps(iterations).encode('utf-8'))

                    finalopinions = iterations[-1]['status']
                    niter = int(iterations[-1]['iteration'])

                    final_opinions[run] = finalopinions
                    final_niter[run] = niter

                    write_dicts(respath, name, max_it, final_opinions, final_niter)

  0%|                                                                             | 2/100000 [00:00<1:33:34, 17.81it/s]

reading dictionaries
final_opinions created
final_iterations created
run 0 epsilon = heterogeneous gamma = 0.0 pm = 0.5 media_op = [0.865708635554234]


100%|████████████████████████████████████████████████████████████████████████| 100000/100000 [4:12:28<00:00,  6.60it/s]


MemoryError: 